# Developing the theme of few-shots

## Init (imports, load models...)

### Necessary imports

In [1]:
from langchain_community.llms.ollama import Ollama

from langchain_community.vectorstores.pgvector import PGVector
from langchain_community.utilities.sql_database import SQLDatabase

from langchain_community.embeddings.ollama import OllamaEmbeddings

from langchain_core.prompts import FewShotPromptTemplate
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

from langchain.chains.sql_database import query

from langchain.agents import Tool
from langchain import agents

from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit 
from langchain_community.agent_toolkits.sql import base

from langchain.prompts.example_selector import SemanticSimilarityExampleSelector

from langchain.globals import set_verbose
from langchain.globals import set_debug
set_debug(True)
set_verbose(True)

from pswrd import PASSWORD_OF_DB
from pswrd import PASSWORD_FOR_VC_CREATOR

### Load models

In [2]:
llama = Ollama(model="llama2:13b", temperature=0)
llama_embeddings = OllamaEmbeddings(model="llama2:13b")
sqlcoder_embeddings = OllamaEmbeddings(model="sqlcoder:15b")
sqlcoder = Ollama(model='sqlcoder:15b', temperature=0)

### Save examples for few-shot prompt

In [3]:
examples = [
    {
        "input": "How many passengers are in the database?",
        "query": "SELECT COUNT(*) FROM public.\"passenger\";"
    },
    {
        "input": "What are the departure times of all flights?",
        "query": "SELECT \"time_out\" FROM public.\"trip\""
    },
    {
        "input": "What is Jane's place?",
        "query": "SELECT \"place\" FROM public.\"pass_in_trip\" JOIN public.\"passenger\" ON public.\"passenger\".\"id\" = public.\"pass_in_trip\".\"passenger\" WHERE public.\"passenger\".\"name\" = \'John\'"
    },
    {
        "input": "Give me all information about airlines",
        "query": "SELECT * FROM public.\"company\""
    },
    {
        "input": "Show me all the trips that are flying out today",
        "query": "SELECT * FROM public.\"trip\"\nWHERE EXTRACT(DAY FROM NOW()) = EXTRACT(DAY FROM \"time_out\")"
    },
    {
        "input": "Which planes depart from Washington?",
        "query": "SELECT \"plane\" FROM public.\"trip\" WHERE \"town_from\" = \'Washington\'"
    },
    {
        "input": "Print out the names of all the planes", 
        "query": "SELECT \"plane\" FROM public.\"trip\""
    },
    {
        "input": "How many people fly on Airbus?",
        "query": "SELECT COUNT(*) FROM public.\"pass_in_trip\" AS paip JOIN public.\"trip\" ON trip.\"id\" = paip.\"trip\" WHERE trip.\"plane\" = \'Airbus\'"
    },
]

### Create prompt

In [4]:
prefix = \
"""You are a silent PostreSQL expert. Given an input question, first create {top_k} syntactically correct PostreSQL query to run, then look at the results and take most correct.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per PostreSQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (\") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date(\'now\') function to get the current date, if the question involves \"today\".
Below are a number of examples of questions and their corresponding SQL queries."""

suffix = \
"""Only use the following tables:
{table_info}

Don\'t explain, use the following format:

User input: {input}
SQL query: """

example_prompt = PromptTemplate.from_template(
    "User input: {input}\nSQL query: {query}"
)

### Connect to DB with Readonly role

In [5]:
CONNECTION_STRING = PGVector.connection_string_from_db_params(
    driver="psycopg2",
    host="localhost",
    port=5433,
    database="llama-test-2",
    user="seq2sql_llama2_rag",
    password=PASSWORD_OF_DB
)

In [6]:
db = SQLDatabase.from_uri(CONNECTION_STRING)

In [7]:
db.run("SET ROLE pg_read_all_data");

#### Check connection

In [8]:
db.run("select * from passenger")

"[(1, 'John'), (2, 'James'), (3, 'Poul'), (4, 'Christofer'), (5, 'Superman')]"

## Simple few-shots by using `create_sql_query_chain`

In [9]:
few_shots_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["input", "table_info", "top_k"],
)

In [10]:
few_shots_chain = query.create_sql_query_chain(llm=llama, db=db, prompt=few_shots_prompt)

In [11]:
res = few_shots_chain.invoke({"question": "Select the names of all the people who are in the airline database"})
print(res)

User input: Select the names of all the people who are in the airline database

SQL query: SELECT name FROM passenger;


In [12]:
db.run("SELECT name FROM passenger")

"[('John',), ('James',), ('Poul',), ('Christofer',), ('Superman',)]"

In [13]:
res = few_shots_chain.invoke({"question": "Print the names of all airlines"})
print(res)

User input: Print the names of all airlines

SQL query: SELECT name FROM company;

This query will retrieve the names of all airlines from the "company" table.


In [14]:
db.run("SELECT name FROM company")

"[('American Airlines',)]"

In [15]:
res = few_shots_chain.invoke({"question": "Print the names of people that end in \"man\""})
print(res)

User input: Print the names of people that end in "man"

SQL query: SELECT name FROM passenger WHERE name LIKE '%man';


In [16]:
db.run("SELECT name FROM passenger WHERE name LIKE \'%man\'")

"[('Superman',)]"

## Few-shots with `SemanticSimilarityExampleSelector`

### Connect to another DB to create embeddings

In [17]:
CONNECTION_STRING = PGVector.connection_string_from_db_params(
    driver="psycopg2",
    host="localhost",
    port=5433,
    database="llama-test",
    user="pgvc_embeddings_creator",
    password=PASSWORD_FOR_VC_CREATOR
)

In [18]:
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    llama_embeddings,
    PGVector,
    k=2,
    connection_string=CONNECTION_STRING,
    pre_delete_collection=True,
    use_jsonb=True
);

In [19]:
few_shots_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["input", "table_info", "top_k"],
)

In [20]:
few_shots_selector_chain = query.create_sql_query_chain(llm=llama, db=db, prompt=few_shots_prompt)

In [21]:
res = few_shots_selector_chain.invoke({
    "question": "Select the names of all the people who are in the airline database", 
    "top_k": "1"
})
print(res)

User input: Select the names of all the people who are in the airline database

SQL query: SELECT name FROM passenger;


In [22]:
db.run("SELECT name FROM passenger")

"[('John',), ('James',), ('Poul',), ('Christofer',), ('Superman',)]"

In [23]:
res = few_shots_selector_chain.invoke({
    "question": "Print the names of all airlines", 
    "top_k": "1"
})
print(res)

User input: Print the names of all airlines

SQL query: SELECT name FROM company;

This query will select all the names of the airlines from the "company" table, and return a list of strings.


In [24]:
db.run("SELECT name FROM company")

"[('American Airlines',)]"

In [25]:
res = few_shots_selector_chain.invoke({
    "question": "Print the names of people that end in \"man\"", 
    "top_k": "1"
})
print(res)

User input: Print the names of people that end in "man"

SQL query: SELECT name FROM passenger WHERE name LIKE '%man';


In [26]:
db.run("SELECT name FROM passenger WHERE name LIKE \'%man\'")

"[('Superman',)]"

## Few-shots selector with SQLCoder

In [27]:
CONNECTION_STRING = PGVector.connection_string_from_db_params(
    driver="psycopg2",
    host="localhost",
    port=5433,
    database="llama-test",
    user="pgvc_embeddings_creator",
    password=PASSWORD_FOR_VC_CREATOR
)

In [38]:
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples=examples,
    embeddings=llama_embeddings,
    vectorstore_cls=PGVector,
    k=2,
    connection_string=CONNECTION_STRING,
    pre_delete_collection=True,
    use_jsonb=True
);

In [39]:
few_shots_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["input", "table_info", "top_k"],
)

In [40]:
few_shots_selector_chain = query.create_sql_query_chain(llm=sqlcoder, db=db, prompt=few_shots_prompt)

In [41]:
res = few_shots_selector_chain.invoke({
    "question": "Select the names of all the people who are in the airline database", 
    "top_k": "1"
})
print(res)

<|endoftext|>


In [42]:
db.run("SELECT name FROM passenger")

"[('John',), ('James',), ('Poul',), ('Christofer',), ('Superman',)]"

In [43]:
res = few_shots_selector_chain.invoke({
    "question": "Print the names of all airlines", 
    "top_k": "1"
})
print(res)

<|endoftext|>


In [44]:
db.run("SELECT name FROM company")

"[('American Airlines',)]"

In [45]:
res = few_shots_selector_chain.invoke({
    "question": "Print the names of people that end in \"man\"", 
    "top_k": "1"
})
print(res)

```
<|endoftext|>


In [46]:
db.run("SELECT name FROM passenger WHERE name LIKE \'%man\'")

"[('Superman',)]"

## Few-shots as tool in agent

In [8]:
CONNECTION_STRING = PGVector.connection_string_from_db_params(
    driver="psycopg2",
    host="localhost",
    port=5433,
    database="llama-test",
    user="pgvc_embeddings_creator",
    password=PASSWORD_FOR_VC_CREATOR
)

In [9]:
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples=examples,
    embeddings=llama_embeddings,
    vectorstore_cls=PGVector,
    k=2,
    connection_string=CONNECTION_STRING,
    pre_delete_collection=True,
    use_jsonb=True
);

In [10]:
toolkit = SQLDatabaseToolkit(llm=sqlcoder, db=db)

example_tool = Tool(
    name="sql_examples",
    func=example_selector.select_examples,
    description="Input this tool a user question to get most semantically similar examples. Always use this tool before generate a query.",
)

sql_suffix=\
"""Begin!

Question: {input}
Thought: I should look at semantically similar examples. 
Then I should look at the tables in the database to see what I can query. 
Then I should query the schema of the most relevant tables.
{agent_scratchpad}"""

agent = base.create_sql_agent(
    llm=llama,
    toolkit=toolkit,
    extra_tools=[example_tool],
    suffix=sql_suffix
)

In [12]:
res = agent.invoke({
    "input": "Select the names of all the people who are in the airline database", 
    "top_k": "1"
})
print(res)



> Entering new LLMChain chain...
Prompt after formatting:

SELECT name FROM passenger WHERE status = 'active'
Double check the postgresql query above for common mistakes, including:
- Using NOT IN with NULL values
- Using UNION when UNION ALL should have been used
- Using BETWEEN for exclusive ranges
- Data type mismatch in predicates
- Properly quoting identifiers
- Using the correct number of arguments for functions
- Casting to the correct data type
- Using the proper columns for joins

If there are any of the above mistakes, rewrite the query. If there are no mistakes, just reproduce the original query.

Output the final SQL query only.

SQL Query: 

> Finished chain.


In [ ]:
res = agent.invoke({
    "input": "Select the names of all the people who are in the airline database", 
})
print(res)